In [1]:
from dataset import ConnTextULDataset
from model import Model
import torch as torch
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package cmudict to /Users/nathan/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [2]:
import pandas as pd
df = pd.read_csv("data/data.csv")

In [16]:
for size in [20, 50, 100, 250, 500, 1000]:
    new_df = df['word_raw'][torch.randint(low=0, high=50000, size=(size,)).tolist()]
    new_df.to_csv(f"data/data_test{size}.csv", index=False)

In [18]:
new_df.to_csv("data/data_test.csv", index=False)

tensor([19766, 44181, 15386, 24120, 19357])

In [ ]:
ds = ConnTextULDataset()
m = Model(len(ds.character_tokenizer),
          len(ds.phonology_tokenizer),
          d_model=64,
          nhead=4)
m.eval();

In [34]:
idx = 10040
num_words = 5
word = ds.words[idx:idx+num_words]
print("word = ", word)
batch = ds[idx:idx+num_words]
#print("batch = ", batch)
orth, phon = batch['orthography'].to('cpu'), batch['phonology'].to('cpu')
print("orth['enc_input_ids'] = ", orth['enc_input_ids'])
print("orth['enc_pad_mask'] = ", orth['enc_pad_mask'])
print("orth['dec_input_ids'] = ", orth['dec_input_ids'])
print("orth['dec_pad_mask'] = ", orth['dec_pad_mask'])
print("phon['enc_input_ids'] = ", phon['enc_input_ids'])
print("phon['enc_pad_mask'] = ", phon['enc_pad_mask'])
print("phon['dec_input_ids'] = ", phon['dec_input_ids'])
print("phon['dec_pad_mask'] = ", phon['dec_pad_mask'])
print("phon['targets'] = ", phon['targets'])

word =  ['sam', 'sat', 'in', 'the', 'sand']
orth['enc_input_ids'] =  tensor([[ 0, 26,  6,  8,  1,  4],
        [ 0, 26,  6, 18,  1,  4],
        [ 0, 11, 21,  1,  4,  4],
        [ 0, 18, 42, 36,  1,  4],
        [ 0, 26,  6, 21, 38,  1]])
orth['enc_pad_mask'] =  tensor([[False, False, False, False, False,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])
orth['dec_input_ids'] =  tensor([[ 0, 26,  6,  8,  4],
        [ 0, 26,  6, 18,  4],
        [ 0, 11, 21,  4,  4],
        [ 0, 18, 42, 36,  4],
        [ 0, 26,  6, 21, 38]])
orth['dec_pad_mask'] =  tensor([[False, False, False, False,  True],
        [False, False, False, False,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])
phon['enc_input_ids'] =  [[tensor([31]), tensor([

In [35]:
z = m(orth_enc_input=orth['enc_input_ids'],
      orth_enc_pad_mask=orth['enc_pad_mask'],
      orth_dec_input=orth['dec_input_ids'],
      orth_dec_pad_mask=orth['dec_pad_mask'],
      phon_enc_input=phon['enc_input_ids'],
      phon_enc_pad_mask=phon['enc_pad_mask'],
      phon_dec_input=phon['dec_input_ids'],
      phon_dec_pad_mask=phon['dec_pad_mask']
      )

In [36]:
print("z['orth'].shape", z['orth'].shape)
print("z['orth'] = ", z['orth'])
print("orth['enc_input_ids'].shape = ", orth['enc_input_ids'].shape)
labels = orth['enc_input_ids'][:,1:]
print("orth['enc_input_ids'][:,1:] = ", labels)
preds = torch.argmax(z['orth'], dim=1)
print("torch.argmax(z['orth'], dim=1) = ", preds)
accuracy = (preds == labels).sum() / len(labels)
print("accuracy = ", accuracy)

z['orth'].shape torch.Size([5, 49, 5])
z['orth'] =  tensor([[[ 0.1693, -0.3138,  0.3945,  0.4023, -0.2128],
         [-0.1762, -0.2139, -0.7959, -0.3276, -0.8687],
         [-0.2521,  0.6397,  0.7131,  0.9635,  0.4533],
         ...,
         [ 0.1149,  0.4284, -0.1474,  1.3147,  0.2178],
         [ 0.9473,  0.4294,  0.6466,  0.5680,  0.0916],
         [ 0.0937,  0.5946,  0.7476,  0.0094,  0.6407]],

        [[ 0.1141, -0.3748,  0.3429,  0.1574, -0.2646],
         [-0.1272, -0.1478, -0.6973, -0.2399, -0.7895],
         [-0.2942,  0.5909,  0.6714,  0.1885,  0.4278],
         ...,
         [ 0.1854,  0.5123, -0.0753,  0.9547,  0.3033],
         [ 0.8905,  0.3622,  0.5781,  0.3989, -0.0041],
         [ 0.2335,  0.7719,  0.8947,  0.1949,  0.8043]],

        [[ 0.2735,  0.1292,  0.4252,  0.1254, -0.1035],
         [-0.2312,  0.3869,  0.0136, -1.3735, -0.9628],
         [-0.2011,  1.1976,  1.2021,  0.7942,  0.4181],
         ...,
         [ 0.1078,  0.1932,  0.0524,  0.8915,  0.2176],
      

In [38]:
preds == labels

tensor([[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [ True, False, False, False, False],
        [False, False, False, False, False]])

In [23]:
print("z['phon'].shape", z['phon'].shape)
print("z['phon'] = ", z['phon'])
torch.argmax(z['phon'], dim=1)

z['phon'].shape torch.Size([1, 2, 4, 33])
z['phon'] =  tensor([[[[ 4.4194e-01,  1.2137e-02, -1.1275e+00, -1.3524e-01,  6.3349e-02,
           -2.1050e-01, -9.0739e-02,  3.0333e-01,  5.6287e-01, -1.5716e-03,
           -4.5574e-01, -8.8808e-01, -7.1200e-01, -3.6442e-01,  7.7940e-02,
           -1.1434e-01,  3.8295e-01, -5.4414e-01,  4.1145e-01,  2.9466e-01,
            3.3463e-01, -2.9968e-02, -7.9627e-01,  1.0003e+00, -1.3511e-01,
            1.0474e+00, -7.4798e-02,  2.0223e-01, -2.5267e-01,  4.4629e-01,
           -2.7202e-02, -4.7697e-01, -4.0632e-01],
          [ 7.3698e-01,  8.1904e-02,  6.9783e-01, -1.5817e-01,  8.8913e-01,
           -3.7205e-01, -1.2592e-01,  1.2707e-01,  1.7307e-01,  2.5315e-01,
           -1.6076e-01,  3.8544e-01, -5.7465e-01, -7.1618e-01, -8.9231e-01,
           -5.9913e-01, -3.8458e-01,  9.2893e-02,  1.3562e-01,  4.3413e-01,
            1.0148e-01,  1.6369e-01, -1.5533e-01,  3.9790e-01, -1.9901e-02,
            3.8274e-01,  9.4589e-02,  4.4247e-01, -7.1658e

tensor([[[1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
          0, 0, 0, 1, 0, 1, 0, 1, 1, 0],
         [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
          0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
         [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
          0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
         [0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
          1, 0, 0, 0, 0, 1, 1, 0, 1, 0]]])

In [10]:
orth

tensor([[0]])

In [ ]:
logits = m(orthography['enc_input_ids'], orthography['enc_pad_mask'],
                orthography['dec_input_ids'], orthography['dec_pad_mask'],
                phonology['enc_input_ids'], phonology['enc_pad_mask'],
                phonology['dec_input_ids'], phonology['dec_pad_mask'])
        
orth_loss = pt.nn.CrossEntropyLoss(ignore_index=4)(logits['orth'], orthography['enc_input_ids'][:,1:]) 
phon_loss = pt.nn.CrossEntropyLoss(ignore_index=2)(logits['phon'], phonology['targets'])
loss = orth_loss + phon_loss

In [57]:
z_prime = m.generate(orth_enc_input=orth, 
                     orth_enc_pad_mask=torch.zeros_like(orth).bool(), 
                     phon_enc_input=phon, 
                     phon_enc_pad_mask=torch.zeros_like(phon).bool(), 
                     deterministic=True)['orth'][:, 1:]
z_prime


phon_output.shape =  torch.Size([1, 1, 64])
phonology_token_logits.shape =  torch.Size([1, 1, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 1, 33])
phon_output.shape =  torch.Size([1, 2, 64])
phonology_token_logits.shape =  torch.Size([1, 2, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 2, 33])
phon_output.shape =  torch.Size([1, 3, 64])
phonology_token_logits.shape =  torch.Size([1, 3, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 3, 33])
phon_output.shape =  torch.Size([1, 4, 64])
phonology_token_logits.shape =  torch.Size([1, 4, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 4, 33])
phon_output.shape =  torch.Size([1, 5, 64])
phonology_token_logits.shape =  torch.Size([1, 5, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 5, 33])
phon_output.shape =  torch.Size([1, 6, 64])
phonology_token_logits.shape =  torch.Size([1, 6, 66])
phonology_token_logits.shape =  torch.Size([1, 2, 6, 33])
phon_output.shape =  torch.Size([1, 7, 64])
phonology_toke

tensor([[16, 40,  4,  0, 16, 15,  5, 45,  7, 31, 19, 16, 42, 40,  8, 15,  0, 16,
         42]])

In [28]:

#assert z.item() == z_prime.item()

################# Check Phonology ########################
z = m(orth, 
      torch.zeros_like(orth).bool(), 
      orth, 
      torch.zeros_like(orth).bool(),
      [[phon][0]], 
      torch.zeros_like(phon).bool(), 
      [[phon][0]], 
      torch.zeros_like(phon).bool()
)['phon']; z


tensor([[[[ 0.6767, -0.1219, -0.5411,  0.1170, -0.3387, -0.1288, -0.5503,
            0.6902,  0.1972, -0.2650,  0.5467,  0.9316,  0.4024, -0.4422,
           -0.1477,  0.7241, -0.9001, -0.2532,  0.5851,  1.1965, -0.7901,
            0.2979, -0.0242,  0.4706,  0.8686,  0.1165,  0.4917,  0.5010,
           -0.1539, -0.0028,  0.1609, -0.2441,  0.3435]],

         [[ 0.5110,  0.4595, -0.6616,  0.0228, -0.6862,  0.1313,  0.8328,
            0.2021,  0.2570,  0.8814,  0.1321, -0.1988, -0.1504, -0.1617,
            0.7846, -0.2460,  0.9513, -0.4330, -0.6739,  0.0562,  0.5938,
            0.0119, -0.3092, -0.0674,  0.9380,  0.5049,  0.0826, -0.9075,
            0.3517,  0.6457,  0.2784, -0.3076,  0.8851]]]],
       grad_fn=<TransposeBackward0>)

In [39]:

z_prime = m.generate(orth, 
                     torch.zeros_like(orth).bool(), 
                     [[phon][0]], 
                     torch.zeros_like(phon).bool(), 
                     deterministic=True)['phon']
z_prime

#assert (torch.where(z[0].argmax(dim=0) == 1)[1] == z_prime[0][1]).all()
print(len(z_prime[0]))

20


In [42]:
B,C,E = 2,3,4
a = torch.rand((B,C,E))

In [44]:
print(a.shape)
print(a.transpose(1,2).shape)

torch.Size([2, 3, 4])
torch.Size([2, 4, 3])


In [62]:
p=torch.rand((1,2,33)); p

tensor([[[0.4301, 0.9611, 0.7723, 0.9204, 0.1864, 0.3543, 0.0298, 0.9956,
          0.0215, 0.8572, 0.2218, 0.4789, 0.0823, 0.7532, 0.1532, 0.0425,
          0.4336, 0.8475, 0.8205, 0.8800, 0.2938, 0.7716, 0.4422, 0.3135,
          0.1729, 0.0260, 0.9826, 0.9753, 0.2033, 0.6105, 0.4312, 0.0927,
          0.9542],
         [0.7559, 0.8657, 0.4542, 0.3938, 0.1533, 0.3608, 0.4802, 0.3396,
          0.7139, 0.2800, 0.1930, 0.4848, 0.7758, 0.4037, 0.2637, 0.3022,
          0.6368, 0.4707, 0.4163, 0.9229, 0.5459, 0.4463, 0.9723, 0.4306,
          0.7044, 0.4951, 0.3131, 0.5019, 0.4948, 0.4797, 0.4198, 0.3487,
          0.9346]]])

In [78]:
torch.bernoulli(p[:,1,:])

tensor([[1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.]])

In [70]:
torch.bernoulli(p)[0]

tensor([[1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.]])

In [80]:
p[:,1]

tensor([[0.7559, 0.8657, 0.4542, 0.3938, 0.1533, 0.3608, 0.4802, 0.3396, 0.7139,
         0.2800, 0.1930, 0.4848, 0.7758, 0.4037, 0.2637, 0.3022, 0.6368, 0.4707,
         0.4163, 0.9229, 0.5459, 0.4463, 0.9723, 0.4306, 0.7044, 0.4951, 0.3131,
         0.5019, 0.4948, 0.4797, 0.4198, 0.3487, 0.9346]])

In [84]:
torch.where(p[:,1] > 0.5)[1]

tensor([ 0,  1,  8, 12, 16, 19, 20, 22, 24, 27, 32])

In [173]:
chkpt = torch.load('models/model0_checkpoint99.pth', map_location=torch.device('cpu'))
m = Model(len(ds.character_tokenizer), len(ds.phonology_tokenizer), d_model=chkpt['d_model'], nhead=chkpt['nhead'])
m.load_state_dict(chkpt['model'])
m.eval();

In [178]:
idx = 10040
word = ds.words[idx:idx+1]
print("word = ", word)
batch = ds[idx:idx+1]
#print("batch = ", batch)
orth, phon = batch['orthography'].to('cpu'), batch['phonology'].to('cpu')
orthography = orth['enc_input_ids']
orthography_mask = orth['enc_pad_mask']
phonology = phon['enc_input_ids']
phonology_mask = phon['enc_pad_mask']

word =  ['sam']


In [179]:
generation = m.generate(orthography, orthography_mask, phonology, phonology_mask, deterministic=False)

In [180]:
ds.character_tokenizer.decode(generation['orth'][:,:].tolist())

['[BOS]ci”γ[EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS][EOS]γ[EOS]γ']

In [181]:
generation['phon'][0]

[tensor([31]),
 tensor([ 9, 14]),
 tensor([14, 15, 17, 26, 28, 29]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([32]),
 tensor([14, 18]),
 tensor([32]),
 tensor([32])]

In [ ]:
ds.phonology_tokenizer.decode(generation['phon'][0])

In [148]:
print(ds.character_tokenizer.vocab)

['[BOS]', '[EOS]', '[CLS]', '[UNK]', '[PAD]', 'i', '’', '\x80', 'o', 'f', 'โ', 'v', '“', '.', ',', 'r', "'", '\x9d', '\x9c', 'g', 'k', 'c', 'n', 'm', 'z', '-', 'y', 'e', 'h', 'j', 'p', 'x', 's', '"', 'q', '\x94', 'ç', '¥', 'w', '”', 'l', 'a', '!', 'u', 'b', 'd', 't', '—', 'γ']


In [158]:
ds

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'character_tokenizer',
 'dataset',
 'phonology_tokenizer',
 'words']